In [1]:
!pip install opencv-python numpy tensorflow keras matplotlib scikit-image pytesseract imutils scipy pandas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pdf2image

In [4]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.9 MB/s eta 0:00:00


In [5]:
# 1. Update package index
!apt-get update

# 2. Fix missing issues and install poppler-utils
!apt-get install -y --fix-missing poppler-utils

# 3. Confirm poppler is installed
!pdftoppm -v


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,270 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,775 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Pac

In [6]:
!pip install pytesseract

In [7]:
import os
import cv2
import numpy as np
from docx import Document
from pdf2image import convert_from_path
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Reshape, Bidirectional, LSTM, Lambda, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pandas as pd
import matplotlib.pyplot as plt

class HandwrittenOCR:
    def __init__(self, student_scripts_path, ground_truth_path):
        self.student_scripts_path = student_scripts_path
        self.ground_truth_path = ground_truth_path

        # Define comprehensive character set
        self.char_list = [''] + sorted(list(
            set("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .,!?;:'\"()-/\\&@#$%^*+=_~<>[]{}|")
        ))

        self.char_to_num = {char: idx for idx, char in enumerate(self.char_list)}
        self.num_to_char = {idx: char for idx, char in enumerate(self.char_list)}

        self.word_images = []
        self.word_labels = []
        self.model = None
        self.prediction_model = None
        self.history = None

    def clean_text(self, text):
        """Ensure text only contains valid characters"""
        return ''.join([c for c in text if c in self.char_to_num])

    def load_ground_truth(self):
        """Load and process all ground truth DOCX files"""
        ground_truth = {}
        for docx_file in os.listdir(self.ground_truth_path):
            if docx_file.endswith('.docx'):
                student_name = os.path.splitext(docx_file)[0].strip()
                try:
                    doc = Document(os.path.join(self.ground_truth_path, docx_file))
                    text = " ".join([para.text for para in doc.paragraphs])
                    ground_truth[student_name] = self.clean_text(text)
                except Exception as e:
                    print(f"Error loading {docx_file}: {e}")
        return ground_truth

    def preprocess_image(self, image):
        """Enhanced image preprocessing pipeline"""
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Adaptive thresholding
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY_INV, 11, 2)

        # Noise removal
        kernel = np.ones((2,2), np.uint8)
        processed = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

        return processed

    def extract_words(self, image, student_name, ground_truth):
        """Improved word extraction with better contour detection"""
        processed = self.preprocess_image(image)

        # Find contours
        contours, _ = cv2.findContours(processed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        boxes = []
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 15 and h > 15 and w*h > 200:  # Filter small noise
                boxes.append((x, y, w, h))

        # Sort boxes left-to-right, top-to-bottom
        boxes = sorted(boxes, key=lambda b: (b[1]//20, b[0]))

        if student_name in ground_truth:
            gt_words = ground_truth[student_name].split()
            for i, (x, y, w, h) in enumerate(boxes[:min(len(boxes), len(gt_words))]):
                # Extract and preprocess word image
                word_img = image[y:y+h, x:x+w]
                word_img = cv2.resize(word_img, (128, 32))  # Wider aspect ratio for words
                word_img = cv2.cvtColor(word_img, cv2.COLOR_BGR2RGB)  # Ensure RGB format

                self.word_images.append(word_img)
                self.word_labels.append(gt_words[i])

    def process_all_scripts(self):
        """Process all student scripts with enhanced error handling"""
        ground_truth = self.load_ground_truth()
        processed_count = 0

        for file in os.listdir(self.student_scripts_path):
            file_path = os.path.join(self.student_scripts_path, file)
            student_name = os.path.splitext(file)[0].strip()

            try:
                if file.lower().endswith('.pdf'):
                    images = convert_from_path(file_path)
                    for img in images:
                        img_np = np.array(img)[..., :3]  # Ensure 3 channels
                        self.extract_words(img_np, student_name, ground_truth)
                        processed_count += 1
                elif file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img = cv2.imread(file_path)
                    if img is not None:
                        self.extract_words(img, student_name, ground_truth)
                        processed_count += 1
            except Exception as e:
                print(f"Error processing {file}: {e}")

        print(f"\nProcessing Complete:")
        print(f"- Processed {processed_count} files")
        print(f"- Extracted {len(self.word_images)} word images")
        print(f"- Ground truth labels: {len(self.word_labels)}")

        # Save dataset info
        self.save_dataset_info()

    def save_dataset_info(self):
        """Save dataset statistics and sample images"""
        os.makedirs("dataset_info", exist_ok=True)

        # Save label distribution
        label_lengths = [len(label) for label in self.word_labels]
        plt.hist(label_lengths, bins=20)
        plt.title("Label Length Distribution")
        plt.savefig("dataset_info/label_lengths.png")
        plt.close()

        # Save character frequency
        char_freq = {}
        for label in self.word_labels:
            for char in label:
                char_freq[char] = char_freq.get(char, 0) + 1
        pd.Series(char_freq).sort_values(ascending=False).to_csv("dataset_info/char_frequency.csv")

        # Save sample images
        os.makedirs("dataset_info/samples", exist_ok=True)
        for i in range(min(20, len(self.word_images))):
            cv2.imwrite(f"dataset_info/samples/sample_{i}.png", cv2.cvtColor(self.word_images[i], cv2.COLOR_RGB2BGR))

    def build_model(self, max_label_length=20):
        """Enhanced CRNN model architecture with configurable sequence length"""
        input_img = Input(shape=(32, 128, 3))  # Matches our new image dimensions

        # CNN Layers
        x = Conv2D(32, (3,3), activation='relu', padding='same')(input_img)
        x = BatchNormalization()(x)
        x = MaxPooling2D((2,2))(x)  # 64x64
        x = Dropout(0.3)(x)

        x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((2,2))(x)  # 32x32
        x = Dropout(0.3)(x)

        x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D((1,2))(x)  # 32x16
        x = Dropout(0.3)(x)

        # Prepare for RNN - Now outputs 32 timesteps (from original width 128)
        x = Reshape((32, 128*4))(x)
        x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2))(x)
        x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2))(x)

        # Output layer
        output = Dense(len(self.char_list), activation='softmax')(x)

        # CTC Loss
        labels = Input(shape=[None], dtype='int32')
        input_length = Input(shape=[1], dtype='int32')
        label_length = Input(shape=[1], dtype='int32')

        loss_out = Lambda(self.ctc_loss, output_shape=(1,), name='ctc')(
            [output, labels, input_length, label_length]
        )

        # Create and compile model
        self.model = Model(
            inputs=[input_img, labels, input_length, label_length],
            outputs=loss_out
        )
        self.model.compile(optimizer=Adam(learning_rate=0.0005),
                         loss={'ctc': lambda y_true, y_pred: y_pred})

        self.prediction_model = Model(input_img, output)
        self.max_label_length = max_label_length

    def ctc_loss(self, args):
        """CTC loss function with sequence length validation"""
        y_pred, labels, input_length, label_length = args

        # Calculate the maximum sequence length
        max_seq_len = K.shape(y_pred)[1]

        # Ensure label lengths don't exceed input lengths
        label_length = K.minimum(label_length, max_seq_len)

        return K.ctc_batch_cost(
            labels,
            y_pred,
            input_length,
            label_length
        )

    def decode_predictions(self, pred):
        input_len = np.ones(pred.shape[0]) * pred.shape[1]
        results = K.get_value(K.ctc_decode(
            pred,
            input_length=input_len,
            greedy=True
        )[0][0])

        output_text = []
        for res in results:
            res = res[res != -1]  # Remove padding
            current_text = ''.join([self.num_to_char.get(int(k), '') for k in res])
            output_text.append(current_text)

        return output_text

    def calculate_accuracy(self, y_true, y_pred):
        """Calculate both word-level and character-level accuracy"""
        word_correct = 0
        char_correct = 0
        char_total = 0

        for true, pred in zip(y_true, y_pred):
            # Word-level accuracy
            if true == pred:
                word_correct += 1

            # Character-level accuracy
            min_len = min(len(true), len(pred))
            for t, p in zip(true[:min_len], pred[:min_len]):
                if t == p:
                    char_correct += 1
                char_total += 1
            char_total += abs(len(true) - len(pred))

        word_acc = word_correct / len(y_true)
        char_acc = char_correct / char_total if char_total > 0 else 0

        return word_acc, char_acc

    def prepare_training_data(self):
        """Prepare data with automatic length adjustment"""
        # Filter out labels that are too long
        filtered_images = []
        filtered_labels = []
        for img, label in zip(self.word_images, self.word_labels):
            if len(label) <= self.max_label_length:
                filtered_images.append(img)
                filtered_labels.append(label)

        print(f"\nFiltered {len(self.word_images)-len(filtered_images)} labels longer than {self.max_label_length} characters")

        X = np.array(filtered_images, dtype=np.float32) / 255.0
        y = [np.array([self.char_to_num[c] for c in word], dtype=np.int32)
             for word in filtered_labels]

        # Pad sequences
        max_label_len = max(len(word) for word in filtered_labels)
        y_padded = np.zeros((len(y), max_label_len), dtype=np.int32)
        for i, seq in enumerate(y):
            y_padded[i, :len(seq)] = seq

        # Calculate sequence lengths - now matches model's output timesteps (32)
        input_length = np.ones((X.shape[0], 1), dtype=np.int32) * 32
        label_length = np.array([[len(word)] for word in filtered_labels], dtype=np.int32)

        return X, y_padded, input_length, label_length

    def train(self, epochs=50, batch_size=32):
        """Enhanced training process with automatic length handling"""
        # Prepare data
        X, y_padded, input_length, label_length = self.prepare_training_data()

        # Split data
        X_train, X_val, y_train, y_val, il_train, il_val, ll_train, ll_val = train_test_split(
            X, y_padded, input_length, label_length, test_size=0.2, random_state=42)

        # Callbacks
        callbacks = [
            EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss'),
            ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss'),
            ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-6, monitor='val_loss')
        ]

        # Train
        self.history = self.model.fit(
            [X_train, y_train, il_train, ll_train],
            np.zeros(X_train.shape[0]),
            validation_data=(
                [X_val, y_val, il_val, ll_val],
                np.zeros(X_val.shape[0])
            ),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=1
        )

        # Calculate final accuracy
        self.evaluate_performance(X_val, y_val)

    def evaluate_performance(self, X_test, y_test):
        """Comprehensive performance evaluation"""
        # Get predictions
        y_pred = self.prediction_model.predict(X_test)
        decoded = self.decode_predictions(y_pred)

        # Convert true labels to text
        y_true_text = []
        for seq in y_test:
            text = ''.join([self.num_to_char.get(int(k), '') for k in seq if k != 0])
            y_true_text.append(text)

        # Calculate accuracies
        word_acc, char_acc = self.calculate_accuracy(y_true_text, decoded)

        print("\nFinal Evaluation:")
        print(f"Word-level Accuracy: {word_acc:.4f}")
        print(f"Character-level Accuracy: {char_acc:.4f}")

        # Save sample predictions
        results = pd.DataFrame({
            'Actual': y_true_text,
            'Predicted': decoded,
            'Correct': [a == p for a, p in zip(y_true_text, decoded)]
        })
        results.to_csv('predictions.csv', index=False)

        # Plot training history
        plt.plot(self.history.history['loss'], label='Training Loss')
        plt.plot(self.history.history['val_loss'], label='Validation Loss')
        plt.title('Training History')
        plt.legend()
        plt.savefig('training_history.png')
        plt.close()

    def predict(self, image_path):
        """Predict text from a new image"""
        img = cv2.imread(image_path)
        if img is None:
            return ""

        # Preprocess
        img = cv2.resize(img, (128, 32))  # Match training dimensions
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = np.expand_dims(img, axis=0) / 255.0

        # Predict
        pred = self.prediction_model.predict(img)
        decoded = self.decode_predictions(pred)

        return decoded[0]

if __name__ == "__main__":
    ocr = HandwrittenOCR(
        student_scripts_path="/content/drive/MyDrive/exam_scripts/project_folder/student_scripts",
        ground_truth_path="/content/drive/MyDrive/exam_scripts/project_folder/ground_truth"
    )

    # Process data and train with increased max label length
    ocr.process_all_scripts()
    ocr.build_model(max_label_length=20)
    ocr.train(epochs=50)




Processing Complete:
- Processed 106 files
- Extracted 23606 word images
- Ground truth labels: 23606

Filtered 0 labels longer than 20 characters
Epoch 1/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 23.7901

591/591 ━━━━━━━━━━━━━━━━━━━━ 38s 45ms/step - loss: 23.7807 - val_loss: 98.3681 - learning_rate: 5.0000e-04
Epoch 2/50
590/591 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 16.2348

591/591 ━━━━━━━━━━━━━━━━━━━━ 27s 46ms/step - loss: 16.2352 - val_loss: 17.7595 - learning_rate: 5.0000e-04
Epoch 3/50
590/591 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 16.2917

591/591 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - loss: 16.2915 - val_loss: 16.3308 - learning_rate: 5.0000e-04
Epoch 4/50
590/591 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 16.1365

591/591 ━━━━━━━━━━━━━━━━━━━━ 41s 47ms/step - loss: 16.1365 - val_loss: 16.1161 - learning_rate: 5.0000e-04
Epoch 5/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 40s 45ms/step - loss: 15.9492 - val_loss: 16.2658 - learning_rate: 5.0000e-04
Epoch 6/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 40s 45ms/step - loss: 16.0558 - val_loss: 16.5841 - learning_rate: 5.0000e-04
Epoch 7/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 15.9921

...

591/591 ━━━━━━━━━━━━━━━━━━━━ 42s 47ms/step - loss: 15.7886 - val_loss: 15.6145 - learning_rate: 2.5000e-04
Epoch 15/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 41s 46ms/step - loss: 15.6677 - val_loss: 15.6190 - learning_rate: 2.5000e-04
Epoch 16/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 40s 45ms/step - loss: 15.5565 - val_loss: 15.6373 - learning_rate: 2.5000e-04
Epoch 17/50
590/591 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 15.4676

591/591 ━━━━━━━━━━━━━━━━━━━━ 42s 47ms/step - loss: 15.4679 - val_loss: 15.5988 - learning_rate: 2.5000e-04
Epoch 18/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 40s 46ms/step - loss: 15.6025 - val_loss: 15.6162 - learning_rate: 2.5000e-04
Epoch 19/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 27s 46ms/step - loss: 15.5932 - val_loss: 15.6153 - learning_rate: 2.5000e-04
Epoch 20/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 42s 49ms/step - loss: 15.4966 - val_loss: 15.6302 - learning_rate: 2.5000e-04
Epoch 21/50
590/591 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 15.6059

591/591 ━━━━━━━━━━━━━━━━━━━━ 25s 41ms/step - loss: 15.6055 - val_loss: 15.5988 - learning_rate: 1.2500e-04
Epoch 22/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 43s 45ms/step - loss: 15.4528 - val_loss: 15.6037 - learning_rate: 1.2500e-04
Epoch 23/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 42s 47ms/step - loss: 15.3998 - val_loss: 15.6048 - learning_rate: 1.2500e-04
Epoch 24/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 15.4468

591/591 ━━━━━━━━━━━━━━━━━━━━ 41s 47ms/step - loss: 15.4468 - val_loss: 15.5976 - learning_rate: 6.2500e-05
Epoch 25/50
590/591 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 15.5327

591/591 ━━━━━━━━━━━━━━━━━━━━ 41s 47ms/step - loss: 15.5323 - val_loss: 15.5874 - learning_rate: 6.2500e-05
Epoch 26/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 41s 47ms/step - loss: 15.3858 - val_loss: 15.6011 - learning_rate: 6.2500e-05
Epoch 27/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 40s 46ms/step - loss: 15.3293 - val_loss: 15.6086 - learning_rate: 6.2500e-05
Epoch 28/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 41s 47ms/step - loss: 15.3502 - val_loss: 15.5990 - learning_rate: 6.2500e-05
Epoch 29/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 41s 47ms/step - loss: 15.3372 - val_loss: 15.6125 - learning_rate: 3.1250e-05
Epoch 30/50
591/591 ━━━━━━━━━━━━━━━━━━━━ 27s 46ms/step - loss: 15.4543 - val_loss: 15.6042 - learning_rate: 3.1250e-05
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step

Final Evaluation:
Word-level Accuracy: 0.0055
Character-level Accuracy: 0.0191
